In [ ]:
library(mlr3verse)
library(mlr3tuning)

## Solution 2: AutoML

### a)

In [ ]:
#| label: 2-a
(task <- tsk("pima"))

### b)

In [ ]:
#| label: 2-b
learners <- list(
    po(lrn("classif.kknn", id = "kknn")), 
    po(lrn("classif.ranger", id = "ranger"))
)

### c)

In [ ]:
#| label: 2-c
# Create a pipeline with preprocessing steps
ppl_preproc <- ppl("robustify", task = task, factors_to_numeric = TRUE)

### d)

In [ ]:
#|label: 2-d
ppl_learners <- ppl("branch", learners)

### e)

In [ ]:
#| label: 2-e
ppl_combined <- ppl_preproc %>>% ppl_learners
plot(ppl_combined)
graph_learner <- as_learner(ppl_combined)

### f)

In [ ]:
#| label: 2-f
# check available hyperparameters for tuning (converting to data.table for 
# better readability)
tail(as.data.table(graph_learner$param_set), 10)

# seeing all our hyperparameters of interest are of type int, we specify the 
# tuning objects accordingly, and dependencies for k and mtry
graph_learner$param_set$values$branch.selection <- 
  to_tune(p_int(1, 2))
graph_learner$param_set$values$kknn.k <- 
  to_tune(p_int(3, 10, depends = branch.selection == 1))
graph_learner$param_set$values$ranger.mtry <-
  to_tune(p_int(1, 5, depends = branch.selection == 2))

# rename learner (otherwise, mlr3 will display a lengthy chain of operations 
# in result tables)
graph_learner$id <- "graph_learner"

### g)

In [ ]:
#| label: 2-g
#| output: false

# make sure to set a seed for reproducible results
set.seed(123)

# perform nested resampling, terminating after 3 evaluations
tuner <- tnr("random_search")

rr <- tune_nested(
  tuner = tuner,
  task = task,
  learner = graph_learner,
  inner_resampling = rsmp("cv", folds = 3),
  outer_resampling = rsmp("cv", folds = 3),
  measure = msr("classif.ce"),
  term_evals = 3
)


### h)

In [ ]:
#| label: 2-h
rr$score()
rr$aggregate()

The performance estimate for our tuned learner then amounts to an MCE of around 0.24.

## Solution 3: Kaggle Challenge

We do not provide an explicit solution here, but have a look at the [tuning code demo](https://github.com/slds-lmu/lecture_i2ml/blob/master/code-demos-pdf/code_demo_kaggle.pdf), which is written in *mlr3* and covers some parts,
and take inspiration from the public contributions on Kaggle.